# L4 — Q&A / RAG over a CSV (Chroma + OpenAI Embeddings)

# Setup

This notebook uses **OpenAI (Python SDK v2) + LangChain v1**.

## Prereqs
1. Set your API key in the environment:

```bash
export OPENAI_API_KEY="..."
```

2. Restart the kernel after setting env vars.


In [1]:
import os

# Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

MODEL = "gpt-5-mini"


We'll index the provided `OutdoorClothingCatalog_1000.csv` and answer questions using retrieval-augmented generation.

In [2]:
from pathlib import Path
import pandas as pd

csv_path = Path("OutdoorClothingCatalog_1000.csv")
if not csv_path.exists():
    # In this repo it may be adjacent; update path if needed.
    csv_path = Path("/mnt/data/OutdoorClothingCatalog_1000.csv")

df = pd.read_csv(csv_path)
df.head()


,Unnamed: 0,name,description
0,0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


## 1) Convert rows to Documents

In [3]:
from langchain_core.documents import Document

docs = []
for _, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    docs.append(Document(page_content=text, metadata={"source": "catalog"}))

len(docs), docs[0].page_content[:200]


(1000,
 "Unnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the ")

## 2) Split + Embed + Store (Chroma)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
splits = splitter.split_documents(docs)

emb = OpenAIEmbeddings(model="text-embedding-3-small")

persist_dir = ".chroma_outdoor_catalog"
vs = Chroma.from_documents(
    documents=splits,
    embedding=emb,
    persist_directory=persist_dir,
)

retriever = vs.as_retriever(search_kwargs={"k": 4})


## 3) RAG chain (retriever + prompt + model)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model=MODEL)
to_text = StrOutputParser()

rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Answer using ONLY the provided context. "
     "If the answer is not in the context, say you don't know."),
    ("user", "Question: {question}\n\nContext:\n{context}")
])

def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | to_text
)

print(rag_chain.invoke("What are some waterproof jackets in the catalog?"))


The catalog includes:

- Outdoor Adventurer Rain Shell — a best-value waterproof rain jacket with TEK waterproof technology, durable laminate interior, three-point adjustable hood, chest and zippered hand pockets; made from 100% recycled nylon and packs into its own pocket.

- TrailGuard Waterproof Gore-Tex Jacket — a waterproof, breathable Gore‑Tex jacket with Gore‑Tex laminate and DWR, three-way adjustable hood, core vents, double-ripstop weave, and it stows in its own pocket.
